In [81]:
from db_info import user, password, host, port, database
import pandas as pd
import mysql.connector

db_connection = mysql.connector.connect(
user = user,
password = password,
host = host,
port = port,
database = database,
)

cursor = db_connection.cursor()

cursor.execute("select * from injury_keyword")

result = cursor.fetchall()

df = pd.DataFrame(result, columns = [i[0] for i in cursor.description])
df.head()

,판례일련번호,사건명,사건번호,선고일자,법원명,사건종류명,판례상세링크,판례정보일련번호,선고,판결유형,...,판결요지,참조조문,참조판례,판례내용,판례내용_상단,판례내용_이유,판례내용_이유_전처리,판례내용_이유_불용어제거,클러스터,키워드
0,241209,법인세등부과처분취소[국내사업장을 가진 외국법인이 국내 및 국외에 걸쳐 선박에 의한 ...,2022두51031,2024.07.25,대법원,세무,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,241209,선고,판결,...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,[1] 구 법인세법(2006. 12. 30. 법률 제8141호로 개정되기 전의 것...,"[3] 대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...","【원고, 상고인 겸 피상고인】 ○○○서비스 (소송대리인 법무법인 케이원챔버 담당변호...",【이 유】 1. 원고의 원심 승소 부분에 대한 상고의 적법 여부에 관한 직권...,{'이유': '1. 원고의 원심 승소 부분에 대한 상고의 적법 여부에 관한 직권판단...,승소 적법 승소 상소 두 취소 승소 여 로써 불복 존재 과세표준 증명책임 사업 연도...,0,None
1,240925,폭행·특수상해(인정된죄명:상해)·특수협박(인정된죄명:협박)·감금[피고인이 위험한 물...,2023도18812,2024.06.13,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,240925,선고,판결,...,"형법 제258조의2 제1항, 제257조 제1항, 제284조, 제283조 제1항은 위...","형법 제257조 제1항, 제258조의2 제1항, 제283조 제1항, 제284조","대법원 2002. 6. 14. 선고 2002도1341 판결, 대법원 2004. 6....",【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,【피 고 인】 피고인【상 고 인】 피고인 및 검사【원심판결】 대전지법 2023. 1...,【이 유】 상고이유를 판단한다. 1. 검사의 상고이유에 관하여 가. 2...,{'이유': '상고이유를 판단한다. 1. 검사의 상고이유에 관하여 가. 2019. ...,폭행 특수 특수 협박 주거지 입금 주기 형광등 값 화가 손 머리 수회 발 옆구리 주...,4,"['보험', '계약', '보험금', '지급', '자동차', '사고', '손해', '..."
2,240809,상해[형사소송법 제33조 제1항 제1호가 정한 필요적 국선변호인 선정사유인 ‘구속’...,2021도6357,2024.05.23,대법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,240809,선고,전원합의체 판결,...,[다수의견] 형사소송법 제33조 제1항 제1호는 피고인에게 변호인이 없는 때에 ...,"헌법 제12조, 형사소송법 제33조 제1항 제1호, 제69조, 제473조 제1항, ...","대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(...",【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,【피 고 인】 피고인【상 고 인】 피고인【변 호 인】 변호사 유소정【원심판결】 인천...,【이 유】 상고이유를 판단한다. 1. 사건의 개요와 쟁점 가. 소송의 진...,{'이유': '상고이유를 판단한다. 1. 사건의 개요와 쟁점 가. 소송의 진행 경과...,개요 쟁점 진행 경과 인천 건조물 침입 죄 제기 징역 구속영장 발부 건 언급 건조물...,0,None
3,219875,주거침입·성폭력범죄의처벌등에관한특례법위반(카메라등이용촬영)·상해,2021노802,2021.11.05,울산지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,219875,선고,판결 : 상고,...,피고인이 별거 상태로 이혼소송 중이던 배우자 甲(女)이 거주하는 원룸에 베란다를...,성폭력범죄의 처벌 등에 관한 특례법 제14조 제1항,None,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,【피 고 인】 피고인【항 소 인】 검사【검 사】 양재헌 외 1인【변 호 인】 ...,【이 유】 1. 이 법원의 심판범위 가. 경합범으로 동시에 기소된 사건에 ...,{'이유': '1. 이 법원의 심판범위 가. 경합범으로 동시에 기소된 사건에 대하여...,경합 동시 기소 개일 분리 일부상소 쌍방 상소 분리 경합 지남 무죄판결 주거침입 벌...,4,"['보험', '계약', '보험금', '지급', '자동차', '사고', '손해', '..."
4,214727,마약류관리에관한법률위반(향정)ㆍ마약류관리에관한법률위반(대마)ㆍ특수상해,2019고단4056,2020.02.19,부산지방법원,형사,/DRF/lawService.do?OC=wanja1996&target=prec&ID...,214727,선고,판결,...,None,None,None,"【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...","【피 고 인】 피고인 1 외 2인【검 사】 김미선, 임홍석(기소), ...",【이 유】범 죄 사 실[범죄전력]피고인 1(2020. 6. 19.자 항소심 판...,{'이유': '범 죄 사 실[범죄전력]피고인 1(2020. 6. 19.자 항소심 판...,죄 사 실 전력 수원지방법원 폭력 감금 죄 징역 집행유예 중앙 마약 류 관리 대마 ...,4,"['보험', '계약', '보험금', '지급', '자동차', '사고', '손해', '..."


# 참조조문 전처리

In [103]:
df['참조조문']=df['참조조문'].fillna('참조조문 없음')


In [104]:
df['참조조문'].value_counts()

참조조문
참조조문 없음                                               803
민법 제750조                                               86
민법 제763조                                               71
자동차손해배상보장법 제3조                                         37
민법 제763조, 민법 제393조                                     30
                                                     ... 
사회보호법 제5조 제1항 , 동법부칙 제2조, 형법 제12조                       1
형법 제37조, 제337조 ,형사소송법 제298조                             1
형법 제335조, 형법 제333조                                      1
특정범죄가중처벌등에관한법률 제5조의5, 특정강력범죄의처벌에관한특례법 제3조, 형법 제35조      1
지방자치법 제102조, 국가배상법 제2조                                  1
Name: count, Length: 2497, dtype: int64

In [105]:
import re
def extract_laws(text):
    if isinstance(text, str):  # 텍스트가 문자열인지 확인
    # '법' 또는 '시행령'으로 끝나는 단어를 정규 표현식으로 추출
        laws = re.findall(r'\b(\w+(?:법|시행령|에 관한 법률|규칙))\b', text)
        return set(laws)
    return set()

# 각 참조조문에서 법과 시행령 이름을 찾기
law_names = set()
for reference in df['참조조문']:
    law_names.update(extract_laws(reference))


# 법과 시행령 이름을 붙이는 함수
def add_law_prefix(x):
    if isinstance(x, str):  
        previous_law = None  # 이전 법 이름 저장
        new_references = []

        for part in x.split(','):
            part = part.strip()
            part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)
            part = re.sub(r'\s*[가-힣]\.\s*', '', part)
            part = part.strip('<br/>')
            part = part.replace('/', ',')
            part = re.sub(r'\s+', ' ', part).strip()

            # 현재 조각에 법 이름이 있는 경우, previous_law 업데이트
            for law_name in law_names:
                if law_name in part:
                    previous_law = law_name  # 이전 법 이름을 현재 법 이름으로 설정
                    break

            # 법 이름이 없고, 조를 포함하는 경우에만 이전 법 이름 붙이기
            if previous_law and not any(law_name in part for law_name in law_names):
                if re.search(r'\d+조', part):  # '조'를 포함하는 경우
                    part = f"{previous_law} {part}"

            new_references.append(part)

        # 결과 조각들을 다시 쉼표로 결합하여 반환
        return ', '.join(new_references)
    return x  # 문자열이 아닐 경우 그대로 반환


In [106]:
# 데이터프레임의 '참조조문' 열에 전처리 적용
df['참조조문'] = df['참조조문'].apply(add_law_prefix)

In [114]:
#조문 참조 정도 확인
refer_clause=[]
for clause in df['참조조문']:
    parts = clause.split(',')
    parts = [part.strip() for part in parts if part.strip()]
    for sub in parts:
        refer_clause.append(sub)
refer_clause = [item for item in refer_clause if item.strip()]
refcla=pd.DataFrame(data=refer_clause)
refcla.value_counts()

0               
참조조문 없음             803
민법 제763조            422
민법 제750조            391
민법 제393조            233
형사소송법 제308조         181
                   ... 
민사소송법 제193조 1항 4      1
민사소송법 제193조 2항        1
민사소송법 제198조           1
민사소송법 제201조           1
회사정리법 제96조            1
Name: count, Length: 3350, dtype: int64

In [113]:
df['참조조문'].value_counts()

참조조문
참조조문 없음                                               803
민법 제750조                                               86
민법 제763조                                               71
자동차손해배상보장법 제3조                                         37
민법 제763조, 민법 제393조                                     30
                                                     ... 
형법 제37조, 형법 제337조, 형사소송법 제298조                          1
형법 제335조, 형법 제333조                                      1
특정범죄가중처벌등에관한법률 제5조의5, 특정강력범죄의처벌에관한특례법 제3조, 형법 제35조      1
근로기준법 제33조, 근로기준법 제40조                                  1
지방자치법 제102조, 국가배상법 제2조                                  1
Name: count, Length: 2473, dtype: int64

# 참조판례 전처리

In [87]:
df['참조판례']=df['참조판례'].fillna('참조판례 없음')
#공백은 '참조판례 없음'으로 대체함.

In [94]:
#참조판례 정리

clean_part=[]
for part in df['참조판례']:
    part = part.strip()
    part = re.sub(r'[\[\]［］].*?[\]\］]', '', part)  # 대괄호와 그 안의 내용 제거
    part = re.sub(r'\s*[가-힣]\.\s*', '', part)  # '가.', '나.', '다.'와 같은 패턴 제거
    part = part.strip('<br/>')  # 공백 제거
    part = part.replace('/', ',')  # '/'를 ','로 변경
    part = re.sub(r'\s+', ' ', part).strip()
    clean_part.append(part)

df['참조판례']=clean_part




In [95]:
df['참조판례']

0       대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2312),...
1       대법원 2002. 6. 14. 선고 2002도1341 판결, 대법원 2004. 6....
2       대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(...
3                                                 참조판례 없음
4                                                 참조판례 없음
                              ...                        
4184                                              참조판례 없음
4185                                              참조판례 없음
4186                                              참조판례 없음
4187                                              참조판례 없음
4188    1966.1.25. 선고 65다2257 판결(판례카아드 1458호, 대법원판결집 1...
Name: 참조판례, Length: 4189, dtype: object

In [99]:
#판례 참조 정도 확인

refered_jud=[]
for clean_part in df['참조판례']:
    parts = re.split(r',\s*(?![^()]*\))', clean_part)
    for sub in parts:
        refered_jud.append(sub)

refered_jud = [item for item in refered_jud if item.strip()]

refjud=pd.DataFrame(data=refered_jud)
refjud.value_counts()

0                                                                          
참조판례 없음                                                                        2281
대법원 2000. 7. 4. 선고 99도4341 판결(공2000하, 1851)                                       9
대법원 1994. 9. 13. 선고 94도1335 판결(공1994하, 2695)                                      8
대법원 2002. 6. 28. 선고 2002도2001 판결(공2002하, 1893)                                    8
대법원 1995. 5. 26. 선고 94다60509 판결(공1995하, 2258)                                     7
                                                                               ... 
대법원 1983.12.27. 선고 83도2557 판결                                                     1
대법원 1983.12.27. 선고 83다카644 판결(공1984,259)                                          1
대법원 1983.12.27 선고 83다카1503 판결                                                     1
대법원 1983.12.13. 선고 83다카975 판결(요특Ⅱ 자동차손해배상보장법 제3조(82)1222면 집31⑥민71 공721호163)       1
형사소송법 제310조(8)1452면)                                                              1


['대법원 1994. 8. 12. 선고 92누12094 판결(공1994하, 2312)',
 '대법원 2021. 2. 25. 선고 2017두237 판결',
 '대법원 2024. 4. 12. 선고 2023두58701 판결(공2024상, 800)',
 '대법원 2002. 6. 14. 선고 2002도1341 판결',
 '대법원 2004. 6. 11. 선고 2004도2018 판결(공2004하, 1198)',
 '대법원 2017. 3. 30. 선고 2017도771 판결',
 '대법원 2009. 5. 28. 선고 2009도579 판결(공2009하, 1060)(변경)',
 '대법원 2011. 9. 29. 선고 2011도10441 판결(변경)',
 '대법원 2011. 11. 10. 선고 2011도12108 판결(변경)',
 '대법원 2012. 6. 28. 선고 2012도5420 판결(변경)',
 '대법원 2016. 6. 9. 선고 2016도4479 판결(변경)',
 '대법원 2017. 1. 12. 선고 2016도19006 판결(변경)',
 '참조판례 없음',
 '참조판례 없음',
 '참조판례 없음',
 '대법원 2010. 5. 27. 선고 2010도3377 판결(공2010하, 1328) ',
 '대법원 1973. 9. 12. 선고 73도1919 판결',
 '대법원 1973. 10. 10. 선고 73도2142 판결',
 '대법원 2011. 2. 10. 선고 2008도4558 판결(공2011상, 593)',
 '참조판례 없음',
 '대법원 2000. 3. 10. 선고 2000도159 판결(공2000상, 1001)',
 '대법원 2004. 4. 27. 선고 2004도482 판결(공2004상, 946)',
 '대법원 2006. 4. 14. 선고 2005도9561 판결(2006상, 836)',
 '참조판례 없음',
 '1970.7.28. 선고 70도1358 판결(요형 형법 제10조(26)16면카9082 집18②형75)',
 '1980.5.27. 선고 80도656 판결(요형 사회보호법 